<a href="https://colab.research.google.com/github/aniparnagian/Artsakh_War_Sentiment-and_LDA_Project-/blob/main/Gensim_Tweets_Aliyev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LDA Topic Modeling with Gensim

Heavily modified and expanded based upon Gensim tutorial

By Jon Chun
28 Mar 2022 Updated

# Setup and Configuration

In [ ]:
!pip install python-slugify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 7.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=f7748b7a31a60a126a54f92b899212b261a2ee618d1eeccc468a3e96a2ab43dd
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=7d1490b3b2577b8073c05275188900ce628fbe7fb19eeff3aebe3c96213add47
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyldavis sklearn


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110 kB 9.3 MB/s 
     |████████████████████████████████| 287 kB 66.5 MB/s 


## Configure Jupyter Notebook

In [ ]:
## Configure Jupyter Notebook

# Ignore warnings

import warnings
warnings.filterwarnings('ignore')

# Enable multiple outputs from one code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
from IPython.display import Image
from ipywidgets import widgets, interactive

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from slugify import slugify

# Get Text Data


## Upload Raw Textfile

In [ ]:
from google.colab import files

TEXT_ENCODING = 'utf-8'

In [10]:
%%time

# NOTE: 1m07s

# Upload Plain Text File
uploaded = files.upload()

# NOTE: Allows for multiple file uploads, will only process the last
#       Left in for future feature addition (processing multiple files at once)
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  text_filename_str = fn

# Extract from Dict and decode binary into char string
text_raw_str = uploaded[text_filename_str].decode(TEXT_ENCODING)

Saving tweets_raw_aliyev_20221012_190820_20221214_234729.csv to tweets_raw_aliyev_20221012_190820_20221214_234729.csv
User uploaded file "tweets_raw_aliyev_20221012_190820_20221214_234729.csv" with length 7347434 bytes
CPU times: user 978 ms, sys: 193 ms, total: 1.17 s
Wall time: 48.4 s


In [11]:
# Recover parameters of original search from filename components
word1, word2, search_str, start_date, start_time, end_date, end_time = text_filename_str.split('_')
end_time = ''.join(end_time.split('.')[:-1])
end_time = ''.join(end_time.split('(')[0])

print(f'word1: {word1}')
print(f'word2: {word2}')
print(f'search_str: {search_str}')
print(f'start_date: {start_date}')
print(f'start_time: {start_time}')
print(f'end_date: {end_date}')
print(f'end_time: {end_time}')

word1: tweets
word2: raw
search_str: aliyev
start_date: 20221012
start_time: 190820
end_date: 20221214
end_time: 234729


In [12]:
# Read raw tweet datafile into DataFrame 'data'

try:
  headers = ['datetime','id','tweet','username','reply_ct','retweet_ct','like_ct','quote_ct','retweeted_id','quoted_id',
             'text_raw','text_noemoji','text_clean','vader','textblob','distilbert'] # ,'roberta']
  dtypes = {'datetime':'str',
            'id':'int',
            'tweet':'string',
            'username':'string',
            'reply_ct':'int',
            'retweet_ct':'int',
            'like_ct':'int',
            'quote_ct':'int',
            'retweeted_id':'string',
            'quoted_id':'string',
            'text_raw':'string',
            'text_noemoji':'string',
            'text_clean':'string',
            'vader':'float',
            'textblob':'float',
            'distilbert':'float'}
            # 'roberta':'float'}
  parse_dates = ['datetime']
  # text_df = pd.read_csv(text_filename_str, skiprows=[0], names=headers, dtype=dtypes, parse_dates=parse_dates, index_col=None)
  text_df = pd.read_csv(text_filename_str, index_col=None)
  text_df.head()
  text_df.info()
except:
  print(f'ERROR: Tweets datafile cannot be read by pd.read_csv')

,datetime,id,tweet,username,reply_ct,retweet_ct,like_ct,quote_ct
0,2022-10-12 19:08:20+00:00,1.58027E+18,@PetrosVon The cousin of Ilham Aliyev,NanaArpi,1.0,0.0,0.0,0.0
1,2022-10-12 19:08:53+00:00,1.58027E+18,@AlinOzinian Aliyev’le mi?,alidile40596400,0.0,0.0,0.0,0.0
2,2022-10-12 19:14:29+00:00,1.58028E+18,BAKU: President Aliyev Blames Armenia for Non-...,HKH77,0.0,0.0,0.0,0.0
3,2022-10-12 19:16:33+00:00,1.58028E+18,@Alkolikyorumlar Aliyev senin kahveden arkadaş...,boran_61,0.0,0.0,1.0,0.0
4,2022-10-12 19:23:04+00:00,1.58028E+18,Azeri mating ritual played out IRL 😂😂😂😂#Azerba...,azergayjan,0.0,0.0,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30001 entries, 0 to 30000
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    30001 non-null  object 
 1   id          30001 non-null  object 
 2   tweet       30001 non-null  object 
 3   username    30000 non-null  object 
 4   reply_ct    30000 non-null  float64
 5   retweet_ct  30000 non-null  float64
 6   like_ct     29999 non-null  float64
 7   quote_ct    29999 non-null  float64
dtypes: float64(4), object(4)
memory usage: 1.8+ MB


## Clean DataFrame

In [13]:
# Drop unused columns

if 'reply_ct' in text_df.columns:
  text_df.drop(columns=['reply_ct'], inplace=True)

if 'retweet_ct' in text_df.columns:
  text_df.drop(columns=['retweet_ct'], inplace=True)

if 'like_ct' in text_df.columns:
  text_df.drop(columns=['like_ct'], inplace=True)

if 'quote_ct' in text_df.columns:
  text_df.drop(columns=['quote_ct'], inplace=True)

if 'retweeted_id' in text_df.columns:
  text_df.drop(columns=['retweeted_id'], inplace=True)

if 'quoted_id' in text_df.columns:
  text_df.drop(columns=['quoted_id'], inplace=True)


In [14]:
# Clean and drop illegal datetime values

text_df['datetime'] = pd.to_datetime(text_df['datetime'], errors='coerce')
text_df = text_df.dropna(subset=['datetime'])
text_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 30000
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   datetime  30000 non-null  datetime64[ns, UTC]
 1   id        30000 non-null  object             
 2   tweet     30000 non-null  object             
 3   username  29999 non-null  object             
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 1.1+ MB


In [15]:
# Drop any row that have at any NaN

text_df.dropna(inplace=True)
text_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29999 entries, 0 to 30000
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   datetime  29999 non-null  datetime64[ns, UTC]
 1   id        29999 non-null  object             
 2   tweet     29999 non-null  object             
 3   username  29999 non-null  object             
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 1.1+ MB


In [16]:
text_df['text_raw'] = text_df['tweet']

text_df['text_raw_type'] = text_df['text_raw'].apply(lambda x: type(x))
text_df['text_raw_type'].value_counts()

<class 'str'>    29999
Name: text_raw_type, dtype: int64

In [17]:
text_df.convert_dtypes()
text_df.info()

,datetime,id,tweet,username,text_raw,text_raw_type
0,2022-10-12 19:08:20+00:00,1.58027E+18,@PetrosVon The cousin of Ilham Aliyev,NanaArpi,@PetrosVon The cousin of Ilham Aliyev,<class 'str'>
1,2022-10-12 19:08:53+00:00,1.58027E+18,@AlinOzinian Aliyev’le mi?,alidile40596400,@AlinOzinian Aliyev’le mi?,<class 'str'>
2,2022-10-12 19:14:29+00:00,1.58028E+18,BAKU: President Aliyev Blames Armenia for Non-...,HKH77,BAKU: President Aliyev Blames Armenia for Non-...,<class 'str'>
3,2022-10-12 19:16:33+00:00,1.58028E+18,@Alkolikyorumlar Aliyev senin kahveden arkadaş...,boran_61,@Alkolikyorumlar Aliyev senin kahveden arkadaş...,<class 'str'>
4,2022-10-12 19:23:04+00:00,1.58028E+18,Azeri mating ritual played out IRL 😂😂😂😂#Azerba...,azergayjan,Azeri mating ritual played out IRL 😂😂😂😂#Azerba...,<class 'str'>
...,...,...,...,...,...,...
29996,2022-12-14 23:41:07+00:00,1.60317E+18,"Now to #Armenia proper. They aren't just the ""...",Proud2BeHate,"Now to #Armenia proper. They aren't just the ""...",<class 'str'>
29997,2022-12-14 23:43:16+00:00,1.60317E+18,@RepKatiePorter Action is needed from the USA....,mystartupangel,@RepKatiePorter Action is needed from the USA....,<class 'str'>
29998,2022-12-14 23:43:17+00:00,1.60317E+18,@layer3xyz @avalancheavax LFG,EminAli21929945,@layer3xyz @avalancheavax LFG,<class 'str'>
29999,2022-12-14 23:47:00+00:00,1.60317E+18,M. @jeromebeguin « très heureux » d’échanger a...,cdcaancfrance,M. @jeromebeguin « très heureux » d’échanger a...,<class 'str'>


<class 'pandas.core.frame.DataFrame'>
Int64Index: 29999 entries, 0 to 30000
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   datetime       29999 non-null  datetime64[ns, UTC]
 1   id             29999 non-null  object             
 2   tweet          29999 non-null  object             
 3   username       29999 non-null  object             
 4   text_raw       29999 non-null  object             
 5   text_raw_type  29999 non-null  object             
dtypes: datetime64[ns, UTC](1), object(5)
memory usage: 1.6+ MB


In [18]:
# Try to convert Tweet text to string (raise error if incompatible encoding)

# text_df['text_raw'] = text_df['text_raw'].apply(lambda x: str(x)) # 
# text_df[['text_raw','username','tweet','text_noemoji','text_clean']] = text_df[['text_raw','username','tweet','text_noemoji','text_clean']].astype('string', errors='raise') # , errors='coerce')
text_df[['text_raw','username','tweet']] = text_df[['text_raw','username','tweet']].astype('string', errors='raise') # , errors='coerce')
# text_df = text_df.dropna(subset=['text_raw'])
text_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29999 entries, 0 to 30000
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   datetime       29999 non-null  datetime64[ns, UTC]
 1   id             29999 non-null  object             
 2   tweet          29999 non-null  string             
 3   username       29999 non-null  string             
 4   text_raw       29999 non-null  string             
 5   text_raw_type  29999 non-null  object             
dtypes: datetime64[ns, UTC](1), object(2), string(3)
memory usage: 1.6+ MB


In [19]:
line_ls = text_df.text_raw.to_list()
print(f'BEFORE filtering out NAType: len(line_ls): {len(line_ls)}')

# line_ls = [x for x in line_ls if isinstance(x, str)]
# print(f' AFTER filtering out NAType: len(line_ls): {len(line_ls)}')

# line_ls = [x for x in line_ls if len(x) > 0]
# print(f' AFTER filtering out null lines: len(line_ls): {len(line_ls)}')

BEFORE filtering out NAType: len(line_ls): 29999


In [20]:
# Agglomerate n Tweets into a Synthetic Document for Topic Modeling

doc_ls = []
tweets_per_doc = 10
doc_current_ls = []
j = 0

for i, aline in enumerate(line_ls):
  
  if (((i % tweets_per_doc) == 0) & (i>0)):
    doc_temp_str = '\n'.join(doc_current_ls)
    doc_ls.append(doc_temp_str)
    print(f'Synthetic Doc #{j}: {doc_temp_str}\n\n')
    j += 1
    # Reset current syn doc
    doc_current_ls = []
    doc_current_ls.append(aline)

  else:
    try:
      doc_current_ls.append(aline)
    except:
      print(f'ERROR: Cannot strip()')

  # line_doc_ls.append(aline)

print('\n\n')

text_raw_str = "\n\n".join(doc_ls)
text_raw_str[:5]


Streaming output truncated to the last 5000 lines.

👉https://t.co/sXZ9dF9nvy https://t.co/hafMQl3VtH
The Russians will still stay in Transcaucasia. Pashinyan will leave together with Putin. Aliyev under sanctions
https://t.co/oJDMaDhR1p
@EldarMamedov4 Azerbaijan is a terrorist regime: victims of aliyev's regime unite
https://t.co/CkilQmEEZo
@BigSn1pez @LKrauss1 @Dictator_Aliyev Don’t forget, genocidal dictator Aliyev’s wife is the vice president. Keeping it all in the family-very normal for a dictatorship. This “professor” can deny as much as he wants, but that only proves his insincerity, his lies. Unethical people gravitate towards each other- Epstein
Aliyev'den Fransa'yı kızdıracak açıklama: Dünya bunu asla unutmamalı https://t.co/BJfPlQqM8y #akit
Türkmənistan manatını götürən bank var?
@BirGun_Gazetesi @ZaferArapkirli Ağanın eli tutulmaz ... Koskoca Aliyev bu ... İki ne yahu ,.. üc bej olsun bari ... :))
Dear Guests!😍
You can add luxury comfort and high quality to your vacation wit

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
doc_ls[-1]
len(doc_ls)

"Stop fascist Azerbaijani dictator aliyev! Demand from him to open the Lachin lifeline road! #StopAzerbaijaniAggression #StopAzerbaijan #Armenians #Artsakh #Karabakh\n@layer3xyz nice\n@layer3xyz Just put on my Santa hat on for Day 1 of 12 Days of @layer3xyz 🎅 #12DaysOfLayer3\n@alibicim https://t.co/Aq4WX6vsbV\n@InvestorSocal The world is not interested in law. The world is not interested in human rights. On the one hand they say democracy and other hand they kill people in wars. And Aliyev reliable partner of Russia and Iran in South Caucasus. And Aliyev is Turkey's brother in South Caucasus.\n#Azerbaijan’s Aliyev no different from #Putin. \n\n#Ukraine #Artsakh #SanctionAliyev https://t.co/UB3GkQDL9Z\n@AzerbaijanMFA @mfa_russia @AzerbaijanMFA this is an aliyev putin agreement to continue the turk Bolshevik destruction of Armenia and the anti Christian anti Armenian policy with the continuation of destroying Armenia selling it to the turks azeris for favors https://t.co/6uL0xAwxwU\n@Sty

2999

# Enter Title

In [22]:
# If this is a Twitter textfile, the search terms are recovered from the filename

# Recover parameters of original search from filename components
try:
  word1, word2, search_str, start_date, start_time, end_date, end_time = text_filename_str.split('_')

  print(f'word1: {word1}')
  print(f'word2: {word2}')
  print(f'search_str: {search_str}')
  print(f'start_date: {start_date}')
  print(f'end_date: {end_date}')
  print('\n')
  
  text_title_str = f'[{search_str}] from [{start_date}] to [{end_date}]'
  print(f'\nTwitter Search String:\n\n  {text_title_str}\n')

except:
  print('This is not a Twitter datafile named in standard format\n\nEnter the Novel information below.')

word1: tweets
word2: raw
search_str: aliyev
start_date: 20221012
end_date: 20221214



Twitter Search String:

  [aliyev] from [20221012] to [20221214]



In [23]:
#@title Enter Title "[Title] by [Author]""

Text_Title = "  [aliyev] from [20221012] to [20221214]" #@param {type:"string"}

# Remove illegal filename punctuation
Text_Title = Text_Title.replace("'","")
Text_Title = Text_Title.replace('"','')
Text_Title = Text_Title.replace('.',' ')

text_title_str = ' '.join([x.capitalize() for x in slugify(Text_Title).replace('-',' ').split()])
text_title_str

'Aliyev From 20221012 To 20221214'


# LDA Model

Introduces Gensim's LDA model and demonstrates its use on the NIPS corpus.


The purpose of this tutorial is to demonstrate how to train and tune an LDA model.

In this tutorial we will:

* Load input data.
* Pre-process that data.
* Transform documents into bag-of-words vectors.
* Train an LDA model.

This tutorial will **not**:

* Explain how Latent Dirichlet Allocation works
* Explain how the LDA model performs inference
* Teach you all the parameters and options for Gensim's LDA implementation

If you are not familiar with the LDA model or how to use it in Gensim, I (Olavur Mortensen)
suggest you read up on that before continuing with this tutorial. Basic
understanding of the LDA model should suffice. Examples:

* `Introduction to Latent Dirichlet Allocation <http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation>`_
* Gensim tutorial: `sphx_glr_auto_examples_core_run_topics_and_transformations.py`
* Gensim's LDA model API docs: :py:class:`gensim.models.LdaModel`

I would also encourage you to consider each step when applying the model to
your data, instead of just blindly applying my solution. The different steps
will depend on your data and possibly your goal with the model.





## Segment Text

In [24]:
text_raw_str[:500]
print('\n')
len(text_raw_str)

'@PetrosVon The cousin of Ilham Aliyev\n@AlinOzinian Aliyev’le mi?\nBAKU: President Aliyev Blames Armenia for Non-Implementation of Tripartite Statement https://t.co/vudVyca3at\n@Alkolikyorumlar Aliyev senin kahveden arkadaşın mı la, Koskoca devlet başkanına ismiyle hitap edecek kadar şuursuzca konuşuyorsun he...\nAzeri mating ritual played out IRL 😂😂😂😂#Azerbaycan #Turkey #karabakhisazerbaijan #aliyev #Erdogan #greywolves https://t.co/14nxnT5UqY\n@HansGutbrod @shafiyev_farid 4 French-Armenian young pr'

5157932

In [25]:
"""
# Split into paragraphs
text_parags_ls = text_raw_str.split('\n')
print(f'There are {len(text_parags_ls)} raw paragraphs\n')

# Delete empty paragraphs
text_parags_ls = [x for x in text_parags_ls if len(x.strip()) > 0]
print(f'There are {len(text_parags_ls)} non-empty paragraphs\n\n')

# Replace stray/embedded /n with a space
text_parags_clean_ls = []
for aparag in text_parags_ls:
  text_parags_clean_ls.append(aparag.replace('\n',' ').strip())

text_parags_ls = text_parags_clean_ls
text_parags_ls[:5] 
"""

text_parags_ls = doc_ls

"\n# Split into paragraphs\ntext_parags_ls = text_raw_str.split('\n')\nprint(f'There are {len(text_parags_ls)} raw paragraphs\n')\n\n# Delete empty paragraphs\ntext_parags_ls = [x for x in text_parags_ls if len(x.strip()) > 0]\nprint(f'There are {len(text_parags_ls)} non-empty paragraphs\n\n')\n\n# Replace stray/embedded /n with a space\ntext_parags_clean_ls = []\nfor aparag in text_parags_ls:\n  text_parags_clean_ls.append(aparag.replace('\n',' ').strip())\n\ntext_parags_ls = text_parags_clean_ls\ntext_parags_ls[:5] \n"

In [26]:
# Verify first paragraph (after title and chapter headings)

text_parags_ls[3]
print('\n')
# Paragraph char count
len(text_parags_ls[3])

"Prime Minister @CMShehbaz met the President of Azerbaijan Ilham Aliyev on the sidelines of #CICASummit2022.\nThey discussed bilateral relations and ways to further enhance cooperation. \n\n#PMShehbazAtCICA \n@AzerbaijanMFA | @AzEmbPak \n@GovtofPakistan https://t.co/Tftj1do28t\nManaf wants to arrest Jalilzadeh in Switzerland and deport him to Azerbaijan. Dictator Aliyev Noldu, was Switzerland a democratic country? Or did it become Aliyev's puppet in that country?#Azerbaijan #Ukraine #Rusia https://t.co/GOQz5LSDjr\n@aliyev_472 @gn_0712 Zehmet olmasa  direkte yaz  izah edecem\nAz önce uluslararsı bir grupla futbol oynadık. Ben Erdoğan'dan, Breziyalı arkadaş Bolsonaro'dan, İtalyan arkadaş Meloni'den, Azeri arkadaş Aliyev'den, Macar arkadaş da Orban'dan şikayet etti. Hepsi de benzer özellikte isimler ve maalesef listeyi uzatmak mümkün... Bat dünya bat!\n@ThomasVLinge Aliyev keeps shelling AM civilians, they keep getting away with warcrimes, so who are we kidding.\nChronicles of Victory: Pr

1771

In [27]:
# Delete any paragraphs shorter than MIN_LEN_PARAG

MIN_LEN_PARAG = 5
MIN_LEN_DOC = 100

# Delete any paragraphs shorter than MIN_LEN_PARAG
text_parags_ls = [x for x in text_parags_ls if len(x) > MIN_LEN_PARAG]

# Trim any leading/trailing/multiple embedded whitespaces
text_parags_ls = [' '.join(x.split()) for x in text_parags_ls]

len(text_parags_ls)

2999

In [28]:
# Agglomerate paragraphs into Documents of MIN_LEN_DOC=1000 chars
"""
parag_ct = len(text_parags_ls)

doc_now_str = ''
doc_now_len = 0
docs = []

for i in range(parag_ct):
  # print(f'Processing Paragraph #{i}')
  parag_now_str = text_parags_ls[i]
  doc_now_str += parag_now_str
  doc_now_len += len(parag_now_str)
  if doc_now_len > MIN_LEN_DOC:
    docs.append(doc_now_str)
    doc_now_str = ''
    doc_now_len = 0

docs[-1] += doc_now_str

print(f'There are now {len(docs)} Documents of {MIN_LEN_DOC} chars or more')
"""

"\nparag_ct = len(text_parags_ls)\n\ndoc_now_str = ''\ndoc_now_len = 0\ndocs = []\n\nfor i in range(parag_ct):\n  # print(f'Processing Paragraph #{i}')\n  parag_now_str = text_parags_ls[i]\n  doc_now_str += parag_now_str\n  doc_now_len += len(parag_now_str)\n  if doc_now_len > MIN_LEN_DOC:\n    docs.append(doc_now_str)\n    doc_now_str = ''\n    doc_now_len = 0\n\ndocs[-1] += doc_now_str\n\nprint(f'There are now {len(docs)} Documents of {MIN_LEN_DOC} chars or more')\n"

In [29]:
# View the first 5 docs

doc_ls[:5]

["@PetrosVon The cousin of Ilham Aliyev\n@AlinOzinian Aliyev’le mi?\nBAKU: President Aliyev Blames Armenia for Non-Implementation of Tripartite Statement https://t.co/vudVyca3at\n@Alkolikyorumlar Aliyev senin kahveden arkadaşın mı la, Koskoca devlet başkanına ismiyle hitap edecek kadar şuursuzca konuşuyorsun he...\nAzeri mating ritual played out IRL 😂😂😂😂#Azerbaycan #Turkey #karabakhisazerbaijan #aliyev #Erdogan #greywolves https://t.co/14nxnT5UqY\n@HansGutbrod @shafiyev_farid 4 French-Armenian young protestors kicked on Azerbaijan's Embassy door in Paris after Azerbaijan BOMBED 36 towns and villages in Armenia, displaced 7,500 people, KILLED 200+ people in 2 days, executed POWs.\n\n❗️To summarize Aliyev's regime's lies and hysteric fake propaganda.\n#French Conservative Journalist @ChdOrnellas said on the French TV CNEWS: “Aliyev said that the inhabitants of Karabakh are citizens of #Azerbaijan, and those who do not like can leave. Right here, he is threatening 120,000 Armenians with f

In [30]:
# View the first 500 chars in the 50th doc

doc_ls[49][:500]

'#EXCLUSIVE : Relations between #Russia and Azerbaijan have reached the highest level - the level of allied interaction. This was stated by #Russian President Vladimir Putin at the meeting with ilham aliyev on the sidelines of #CICASummit2022 in #Astana. https://t.co/5PHrvaMccj https://t.co/TQHLlQ7XUU\n#EXCLUSIVE : #Russian President Vladimir Putin at a meeting with Aliyev : "Naturally, we always keep an eye on issues related to the settlement of the situation in the region. I hope we will use thi'

In [31]:
# View the last 100 chars in the last doc

doc_ls[-1][-100:]

'v’s #Azerbaijan treats creating humanitarian disasters as just that - Sport. https://t.co/PHGa2tKGmt'

## Pre-process and vectorize the documents

As part of preprocessing, we will:

* Expand Contractions
* Tokenize (split the documents into tokens)
* Define stopwords and filter out
* Lemmatize the tokens.
* Compute bigrams.
* Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We
remove numeric tokens and tokens that are only a single character, as they
don't tend to be useful, and the dataset contains a lot of them.

.. Important::

   This tutorial uses the nltk library for preprocessing, although you can
   replace it with something else if you want.




### Steps

#### Expand Contractions

In [54]:
# Expand Contractions (e.g. can't -> can not)

import contractions

# Test
contractions.fix("yall're happy now it isn't a don't tell problem", slang=False) # default: true

"yall're happy now it is not a do not tell problem"

In [55]:
doc_ls[:5]

[['petrosvon',
  'the',
  'cousin',
  'of',
  'ilham',
  'aliyev',
  'alinozinian',
  'aliyev',
  'le',
  'mi',
  'baku',
  'president',
  'aliyev',
  'blames',
  'armenia',
  'for',
  'non',
  'implementation',
  'of',
  'tripartite',
  'statement',
  'https',
  't',
  'co',
  'vudvyca3at',
  'alkolikyorumlar',
  'aliyev',
  'senin',
  'kahveden',
  'arkadaşın',
  'mı',
  'la',
  'koskoca',
  'devlet',
  'başkanına',
  'ismiyle',
  'hitap',
  'edecek',
  'kadar',
  'şuursuzca',
  'konuşuyorsun',
  'he',
  'azeri',
  'mating',
  'ritual',
  'played',
  'out',
  'irl',
  'azerbaycan',
  'turkey',
  'karabakhisazerbaijan',
  'aliyev',
  'erdogan',
  'greywolves',
  'https',
  't',
  'co',
  '14nxnt5uqy',
  'hansgutbrod',
  'shafiyev_farid',
  '4',
  'french',
  'armenian',
  'young',
  'protestors',
  'kicked',
  'on',
  'azerbaijan',
  's',
  'embassy',
  'door',
  'in',
  'paris',
  'after',
  'azerbaijan',
  'bombed',
  '36',
  'towns',
  'and',
  'villages',
  'in',
  'armenia',
  'd

In [56]:
len(doc_ls)

2999

In [57]:
# Expand all Contractions paragraph by paragraph

"""
docs_clean_ls = []
for adoc in doc_ls:
  docs_clean_ls.append(contractions.fix(adoc))

docs = docs_clean_ls
docs[:5]
""";

docs = doc_ls

'\ndocs_clean_ls = []\nfor adoc in doc_ls:\n  docs_clean_ls.append(contractions.fix(adoc))\n\ndocs = docs_clean_ls\ndocs[:5]\n'

#### Tokenize Text

In [59]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

AttributeError: ignored

In [ ]:
# Verify docs

# content and token count of first doc
docs[0]
print(f'\n\nThere are {len(docs[0])} tokens in the first document')

#### Customize Stopwords

In [60]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords_ls = stopwords.words('english')

print(f'\nThe first ten stopwords:')
stopwords_ls[20:]
print(f'\n\nThere are [{len(stopwords_ls)}] English stopwords imported from NLTK')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True


The first ten stopwords:


['himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don



There are [179] English stopwords imported from NLTK


In [61]:
# Define list of stopwords listed above that we should keep

stopword_keep_ls = []

"""
stopword_keep_ls = [
    'against',
    'not',
    'no',
    'nor',

    'do',
    'are',
    'is',
    'are',
    "not",
    'could',
    'did',
    'have',
    'has',
    'had',
    'might',
    'must',
    'need',
    'shall',
    'should',
    'was',
    'were',
    'would',

    'own',
    'same',
    'how',
    'why'
]
"""

'\nstopword_keep_ls = [\n    \'against\',\n    \'not\',\n    \'no\',\n    \'nor\',\n\n    \'do\',\n    \'are\',\n    \'is\',\n    \'are\',\n    "not",\n    \'could\',\n    \'did\',\n    \'have\',\n    \'has\',\n    \'had\',\n    \'might\',\n    \'must\',\n    \'need\',\n    \'shall\',\n    \'should\',\n    \'was\',\n    \'were\',\n    \'would\',\n\n    \'own\',\n    \'same\',\n    \'how\',\n    \'why\'\n]\n'

In [62]:
# Derive final stopword list

stopword_keep_ls = list(set(stopwords_ls) - set(stopword_keep_ls))
stopword_keep_ls
print('\n\n')
print(f'BEFORE: {len(stopwords_ls)} stopwords')
print(f' AFTER: {len(stopword_keep_ls)}')

['herself',
 'up',
 'by',
 'in',
 'hasn',
 'very',
 'here',
 'now',
 'hadn',
 'won',
 'wasn',
 'because',
 'an',
 'under',
 'over',
 'y',
 'through',
 't',
 'same',
 'am',
 'than',
 "haven't",
 'there',
 'which',
 'and',
 'those',
 'too',
 'your',
 'other',
 'with',
 'below',
 'both',
 'are',
 "she's",
 'will',
 'after',
 'we',
 'until',
 'weren',
 'again',
 'she',
 'shan',
 'only',
 "you're",
 'her',
 "wasn't",
 'itself',
 "should've",
 'whom',
 "aren't",
 'shouldn',
 'what',
 'myself',
 'into',
 'i',
 'how',
 'few',
 "hadn't",
 'this',
 'at',
 'can',
 'about',
 'on',
 'further',
 'to',
 'as',
 'its',
 'between',
 'down',
 'doesn',
 'ourselves',
 'the',
 'once',
 'some',
 'they',
 'who',
 "shouldn't",
 "weren't",
 'own',
 'nor',
 'haven',
 'didn',
 "don't",
 'mustn',
 'yours',
 'have',
 "isn't",
 'ma',
 "wouldn't",
 "shan't",
 'theirs',
 'that',
 'doing',
 'off',
 'for',
 'above',
 's',
 'being',
 'me',
 'our',
 'but',
 'm',
 'their',
 'when',
 'should',
 "needn't",
 'o',
 'no',
 'you




BEFORE: 179 stopwords
 AFTER: 179


In [63]:
# Remove all stopwords with contractions (optional - could also remove fragments like 'll', 're', and 've' but not necessary)

stopwords_ls = [x for x in stopword_keep_ls if not "'" in x]
stopwords_ls[20:]
print(f'\n\nThere are now {len(stopwords_ls)} stopwords after removing words with contractions')


['than',
 'there',
 'which',
 'and',
 'those',
 'too',
 'your',
 'other',
 'with',
 'below',
 'both',
 'are',
 'will',
 'after',
 'we',
 'until',
 'weren',
 'again',
 'she',
 'shan',
 'only',
 'her',
 'itself',
 'whom',
 'shouldn',
 'what',
 'myself',
 'into',
 'i',
 'how',
 'few',
 'this',
 'at',
 'can',
 'about',
 'on',
 'further',
 'to',
 'as',
 'its',
 'between',
 'down',
 'doesn',
 'ourselves',
 'the',
 'once',
 'some',
 'they',
 'who',
 'own',
 'nor',
 'haven',
 'didn',
 'mustn',
 'yours',
 'have',
 'ma',
 'theirs',
 'that',
 'doing',
 'off',
 'for',
 'above',
 's',
 'being',
 'me',
 'our',
 'but',
 'm',
 'their',
 'when',
 'should',
 'o',
 'no',
 'you',
 'it',
 'he',
 'aren',
 'where',
 'couldn',
 'from',
 'yourself',
 'then',
 'so',
 'such',
 'ours',
 'themselves',
 'll',
 're',
 'why',
 'each',
 'against',
 'had',
 'more',
 'of',
 'been',
 'wouldn',
 've',
 'himself',
 'be',
 'yourselves',
 'is',
 'were',
 'my',
 'having',
 'any',
 'him',
 'did',
 'isn',
 'does',
 'needn',
 'a



There are now 153 stopwords after removing words with contractions


**[CUSTOMIZE] Stopwords list by adding or deleting tokens**

In [64]:
# [CUSTOMIZE] Stopwords to ADD or DELETE from default NLTK English stopword list

STOPWORDS_ADD_SET = set(['bazinga', 'woo-hoo']) # Edit this list to add new words to the stopwords list
STOPWORDS_DEL_SET = set(['the']) # Edit this list to remove exising words from the stopwords list

stopwords_en_ls = list(set(stopwords_ls).difference(set(STOPWORDS_DEL_SET)).union(set(STOPWORDS_ADD_SET)))
print(f'Final Count after Customized Add/Del: {len(stopwords_en_ls)} Stopwords')

Final Count after Customized Add/Del: 154 Stopwords


#### Lemmatize

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a
stemmer in this case because it produces more readable words. Output that is
easy to read is very desirable in topic modelling.




In [65]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [66]:
%%time

# NOTE: 0m24s @03:48 on 20220228 Colab Pro 

# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

CPU times: user 2.88 s, sys: 4.07 ms, total: 2.88 s
Wall time: 2.91 s


We find bigrams in the documents. Bigrams are sets of two adjacent words.
Using bigrams we can get phrases like "machine_learning" in our output
(spaces are replaced with underscores); without bigrams we would only get
"machine" and "learning".

Note that in the code below, we find bigrams and then add them to the
original data, because we would like to keep the words "machine" and
"learning" as well as the bigram "machine_learning".

.. Important::
    Computing n-grams of large dataset can be very computationally
    and memory intensive.




### Identify Bi- and Tri-Grams

In [45]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

We remove rare words and common words based on their *document frequency*.
Below we remove words that appear in less than 20 documents or in more than
50% of the documents. Consider trying to remove words only based on their
frequency, or maybe combining that with this approach.




In [46]:
#@markdown Minimum number of documents a word must appear in: (default 20)

Min_No_Documents = 5 #@param {type:"slider", min:5, max:100, step:1}

#@markdown Max percent of documents a word can appear in: (default 0.50 or 50%)

Max_Percent_Documents = 0.49 #@param {type:"slider", min:0.05, max:0.9, step:0.01}


In [47]:
# docs_ls_ls = [astring.split() for astring in docs]
# docs = docs_ls_ls

In [48]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=Min_No_Documents, no_above=Max_Percent_Documents)

Finally, we transform the documents to a vectorized form. We simply compute
the frequency of each word, including the bigrams.




In [49]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

Let's see how many tokens and documents we have to train on.




In [50]:
# Corpus as a list of documents, each a list of tokens identifyed by dictionary tuples

len(corpus)

2999

In [51]:
type(corpus[0])

list

In [52]:
corpus[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 2),
 (5, 2),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1)]

In [53]:
print('Number of unique tokens: %d' % len(dictionary))  # Orig 1864
print('Number of documents: %d' % len(corpus))          # Orig 1740

Number of unique tokens: 12081
Number of documents: 2999


## Training

We are ready to train the LDA model. We will first discuss how to set some of
the training parameters.

First of all, the elephant in the room: how many topics do I need? There is
really no easy answer for this, it will depend on both your data and your
application. I have used 10 topics here because I wanted to have a few topics
that I could interpret and "label", and because that turned out to give me
reasonably good results. You might not need to interpret all your topics, so
you could use a large number of topics, for example 100.

``chunksize`` controls how many documents are processed at a time in the
training algorithm. Increasing chunksize will speed up training, at least as
long as the chunk of documents easily fit into memory. I've set ``chunksize =
2000``, which is more than the amount of documents, so I process all the
data in one go. Chunksize can however influence the quality of the model, as
discussed in Hoffman and co-authors [2], but the difference was not
substantial in this case.

``passes`` controls how often we train the model on the entire corpus.
Another word for passes might be "epochs". ``iterations`` is somewhat
technical, but essentially it controls how often we repeat a particular loop
over each document. It is important to set the number of "passes" and
"iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable
logging (as described in many Gensim tutorials), and set ``eval_every = 1``
in ``LdaModel``. When training the model look for a line in the log that
looks something like this::

   2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set ``passes = 20`` you will see this line 20 times. Make sure that by
the final passes, most of the documents have converged. So you want to choose
both passes and iterations to be high enough for this to happen.

We set ``alpha = 'auto'`` and ``eta = 'auto'``. Again this is somewhat
technical, but essentially we are automatically learning two parameters in
the model that we usually would have to specify explicitly.




In [67]:
#@markdown How many Topics do you want to find?

No_of_Topics = 10 #@param {type:"slider", min:2, max:200, step:1}

#@markdown Default 10-50 depending upon how large the text and diverse the vocabulary

In [68]:
%%time

# NOTE: 2m01s @20220328 with Google Colab/CPU on Harry Potter

# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = No_of_Topics
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

CPU times: user 59.8 s, sys: 628 ms, total: 1min
Wall time: 1min


We can compute the topic coherence of each topic. Below we display the
average topic coherence and print the topics in order of topic coherence.

Note that we use the "Umass" topic coherence measure here (see
:py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
obtained an implementation of the "AKSW" topic coherence measure (see
accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can
see that the topics below make a lot of sense. However, they are not without
flaws. We can see that there is substantial overlap between some topics,
others are hard to interpret, and most of them have at least some terms that
seem out of place. If you were able to do better, feel free to share your
methods on the blog at http://rare-technologies.com/lda-training-tips/ !




In [69]:
%whos str

Variable            Type    Data/Info
-------------------------------------
TEXT_ENCODING       str     utf-8
Text_Title          str       [aliyev] from [20221012] to [20221214]
aline               str     In der Diktatur #aserbaid<...>. https://t.co/zTGfSc5Avo
doc_temp_str        str     Stop fascist Azerbaijani <...>. https://t.co/PHGa2tKGmt
end_date            str     20221214
end_time            str     234729.csv
fn                  str     tweets_raw_aliyev_2022101<...>90820_20221214_234729.csv
search_str          str     aliyev
start_date          str     20221012
start_time          str     190820
temp                str     abdiraufova
text_filename_str   str     tweets_raw_aliyev_2022101<...>90820_20221214_234729.csv
text_raw_str        str     @PetrosVon The cousin of <...>. https://t.co/PHGa2tKGmt
text_title_str      str     Aliyev From 20221012 To 20221214
token               str     phga2tkgmt
word1               str     tweets
word2               str     raw


In [70]:
search_str

'aliyev'

In [71]:
top_topics = model.top_topics(corpus) #, num_words=20)


# from pprint import pprint
# pprint(top_topics)

# print('\n\n')
# for i,atopic in enumerate(top_topics):
#   atopic_coherence_str = f'Topic #{i}: coherence = {atopic[1]}'
#   print(atopic_coherence_str)
#   report_line_ls.append(atopic_coherence_str)


topic_word_ls = []

topic_word_ls.append(text_title_str)

# print(text_title_str)

for i, atopic in enumerate(top_topics):
  topic_str = f'\n\nTopic #{i}: {atopic[1]} (coherence)'
  # print(topic_str)
  topic_word_ls.append(topic_str)
  for j, aword in enumerate(atopic[0]):
    word_str = f'  Word #{j}: {aword}'
    # print(word_str)
    topic_word_ls.append(word_str)

avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
avg_topic_coherence_str = f'Average topic coherence: {avg_topic_coherence:.4f}'
# print(avg_topic_coherence_str)
topic_word_ls.append(f'\n\n{avg_topic_coherence_str}')

topic_word_str = "\n".join(topic_word_ls)
print(topic_word_str)

# Save to file
filename_topic_word_str = f'lda_topic_{search_str}_word_report.txt'
with open(filename_topic_word_str, 'w') as fp:
  fp.write(topic_word_str)

files.download(filename_topic_word_str)

Aliyev From 20221012 To 20221214


Topic #0: -0.9629506110539887 (coherence)
  Word #0: (0.009746824, 'armenia')
  Word #1: (0.009085524, 'not')
  Word #2: (0.00725783, 'will')
  Word #3: (0.0056899, 'he')
  Word #4: (0.0056095026, 'wa')
  Word #5: (0.0055580083, 'this')
  Word #6: (0.0054326314, 'armenian')
  Word #7: (0.004953795, 'from')
  Word #8: (0.0049035605, 'are')
  Word #9: (0.004898979, 'azerbaijani')
  Word #10: (0.00487096, 'be')
  Word #11: (0.0046649543, 'by')
  Word #12: (0.0044845208, 'you')
  Word #13: (0.0043818867, 'we')
  Word #14: (0.0043516043, 'ha')
  Word #15: (0.0043278285, 'no')
  Word #16: (0.004305844, 'a')
  Word #17: (0.004210641, 'iran')
  Word #18: (0.004093011, 'an')
  Word #19: (0.0038625605, 'da')


Topic #1: -0.9788138979588117 (coherence)
  Word #0: (0.012117436, 'you')
  Word #1: (0.010802241, 'are')
  Word #2: (0.0087307375, 'armenian')
  Word #3: (0.007754231, 'by')
  Word #4: (0.0075167227, 'his')
  Word #5: (0.0071010827, 'this')
  Word #6: (0

7656

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Visualize

In [72]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable


In [73]:
# vis_data = gensimvis.prepare(lda, corpus, dictionary)
vis_data = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(vis_data)

## Things to experiment with

* ``no_above`` and ``no_below`` parameters in ``filter_extremes`` method.
* Adding trigrams or even higher order n-grams.
* Consider whether using a hold-out set or cross-validation is the way to go for you.
* Try other datasets.

## Where to go from here

* Check out a RaRe blog post on the AKSW topic coherence measure (http://rare-technologies.com/what-is-topic-coherence/).
* pyLDAvis (https://pyldavis.readthedocs.io/en/latest/index.html).
* Read some more Gensim tutorials (https://github.com/RaRe-Technologies/gensim/blob/develop/tutorials.md#tutorials).
* If you haven't already, read [1] and [2] (see references).

## References

1. "Latent Dirichlet Allocation", Blei et al. 2003.
2. "Online Learning for Latent Dirichlet Allocation", Hoffman et al. 2010.


